In [ ]:
import getpass # Denne funksjonen lar deg skrive inn passordet med et input felt, istedenfor å skrive det inn i notebooken
passord = getpass.getpass()

In [ ]:
import pandas as pd
import sqlite3
from sqlalchemy import Table, create_engine
from sqlalchemy.sql import select
from flask_sqlalchemy import SQLAlchemy
import getpass
import cx_Oracle
import json
import os

from models.models_delt import connect

In [ ]:
with open("config.json") as config: # Laster in valg fra config.json
    config = json.load(config)

conn, engine, db = connect()

In [ ]:
bruker = getpass.getuser()[:3]
db = cx_Oracle.connect(bruker,passord, "DB1P")

In [ ]:
delreg = config["perioder"]["t"]["delreg"]
df_feil = pd.read_sql(f"SELECT DELREG_NR, ENHETS_TYPE, ENHETS_ID, SKJEMA, LOPENR, FELT_ID, FEILID, FTEKST, BRUKER, INT_ID, RAD_NR FROM DYNAREV.VW_KONTROLL_FEIL WHERE DELREG_NR='{delreg}'", con = db)
df_feil["DELREG_NR"] = df_feil["DELREG_NR"].astype(str)

# Nedenfor fjernes de som er reviderte
enheter = pd.read_sql(f"SELECT ENHETS_ID, REV_KODE, INN_DATO, REV_DATO FROM DYNAREV.VW_SKJEMA_ENHET WHERE DELREG_NR = {delreg}", con=db)
enheter = list(enheter.loc[enheter["REV_KODE"] != "R"]["ENHETS_ID"].unique())
df_feil = df_feil.loc[df_feil["ENHETS_ID"].isin(enheter)]

In [ ]:
dff = pd.read_sql(f'SELECT * FROM {config["tabeller"]["raadata"]}', con=engine)

In [ ]:
kolonner = dff.columns.difference(df_feil.columns).tolist()
kolonner += ["ENHETS_ID"]

df = pd.merge(df_feil, dff[kolonner], on= "ENHETS_ID", how = "right")

df = df.dropna(subset = ["feilliste"])
df = df.drop_duplicates(subset = ["ORGNR", "feilliste", "FELT_ID"])

In [ ]:
df["kommentar"] = ""
df["feilliste"] = df["FTEKST"]

In [ ]:
df.to_csv(f"{config['data']['filsti']}/feilliste.csv", index = False)

In [ ]:
for i in df_feil["FTEKST"].unique():
    print(i)
    print("---------")